In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install -r requirements.txt
from src.preprocessing import DataPreprocessor
from src.model import Model
from src.gui import *

In [ ]:
# Load training and testing datasets
dataset = DataPreprocessor(
    filepath="data/Line Listing(dxjZy).xlsx",
    column_names=['Case ID', 'Suspect Product Active Ingredients', 'Serious', 'Sex', 'Patient Age', 'Patient Weight']
)
dataset.explode_column("Suspect Product Active Ingredients", ";")

print("Value Counts:")
print(dataset.get_value_counts("Serious"))

print("\nData Types:")
print(dataset.get_dataframe().dtypes)

dataset.get_dummies(["Suspect Product Active Ingredients"])
dataset.data.groupby("Case ID").max().reset_index()
dataset.drop_columns(["Case ID"])

dataset.ensure_numeric_column("Patient Age", decimal=False)

dataset.convert_nulls("Patient Weight", nulls=["Not Specified"], output="0 KG")
dataset.ensure_numeric_column("Patient Weight")

dataset.encode_column("Sex")
dataset.encode_column("Serious")

dataset.drop_all_nulls()

print("\nNew Value Counts:")
print(dataset.get_value_counts("Serious"))

x_train, x_test, y_train, y_test = dataset.get_standardised_train_test_split(
    dataset.data.columns, 
    y="Serious", 
    test_size=0.2, 
    random_state=42
)

In [ ]:
# Running model
input_dim = len(x_train[0])
model = Model(input_dim)
model.train_model(x_train, y_train)
model.evaluate_model(x_test, y_test)